In [67]:
import geopandas as gpd
import pandas as pd
import numpy as np
from numpy import average
from shapely.geometry import Point
import matplotlib.pyplot as plt
import folium
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster
import statistics


### Most updated USWTDB version 3

In [2]:
wind = gpd.read_file('../data/USWTDB/uswtdb_v3_2_20201014.geojson')

In [3]:
wind.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 65548 entries, 0 to 65547
Data columns (total 18 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   case_id     65548 non-null  float64 
 1   t_state     65548 non-null  object  
 2   p_name      65548 non-null  object  
 3   p_year      64055 non-null  float64 
 4   p_tnum      65548 non-null  float64 
 5   p_cap       60580 non-null  float64 
 6   t_manu      59569 non-null  object  
 7   t_model     59245 non-null  object  
 8   t_cap       59571 non-null  float64 
 9   t_hh        58629 non-null  float64 
 10  t_rd        58911 non-null  float64 
 11  t_rsa       58911 non-null  float64 
 12  t_ttlh      58629 non-null  float64 
 13  t_conf_atr  65548 non-null  float64 
 14  t_conf_loc  65548 non-null  float64 
 15  xlong       65548 non-null  float64 
 16  ylat        65548 non-null  float64 
 17  geometry    65548 non-null  geometry
dtypes: float64(13), geometry(1), object(4)

In [49]:
manufacturers = wind.t_manu.value_counts().to_frame().reset_index()
manufacturers['index'].iloc[40:68]
manufacturers.head(20)

,index,t_manu
0,GE Wind,25858
1,Vestas,13904
2,Siemens,5144
3,Gamesa,3076
4,Mitsubishi,2791
5,Suzlon,1316
6,Siemens Gamesa Renewable Energy,1212
7,Nordex,1145
8,Acciona,758
9,Clipper,681


In [5]:
wind.shape

(65548, 18)

In [6]:
year_install = wind.p_year.value_counts().to_frame().reset_index()

In [7]:
year_install =  year_install.sort_values(by = 'index')

In [8]:
year_install #wind turbines by installation year still operating

,index,p_year
36,1981.0,11
17,1982.0,937
22,1983.0,433
30,1984.0,155
19,1985.0,714
31,1986.0,151
28,1987.0,213
26,1988.0,277
25,1989.0,288
24,1990.0,327


### US Wind Turbine Data from 2014

In [9]:
wind_old = pd.read_csv('../data/Onshore_Industrial_Wind_Turbine_Locations_for_the_United_States_to_March_2014.csv')

In [10]:
wind_old.columns

Index(['FID', 'unique_id', 'site_name', 'total_turb', 'on_year', 'year_range',
       'on_year_s', 'manufac', 'model', 'type_tower', 'decommiss',
       'MW_turbine', 'total_cpcy', 'total_ht', 'tower_h', 'blade_l',
       'rotor_dia', 'rotor_s_a', 'lat_DD', 'long_DD', 'state', 'county',
       'conf_attr', 'conf_loc', 'WENDI_name', 'EIA_name', 'FAA_jdate',
       'FAA_AGL', 'FAA_ORS', 'image_name', 'image_year', 'comments'],
      dtype='object')

In [11]:
wind_old['geometry'] = wind_old.apply(lambda x: Point((float(x.long_DD), 
                                                         float(x.lat_DD))), 
                                        axis=1)

In [12]:
wind_old.head(1)

,FID,unique_id,site_name,total_turb,on_year,year_range,on_year_s,manufac,model,type_tower,...,conf_loc,WENDI_name,EIA_name,FAA_jdate,FAA_AGL,FAA_ORS,image_name,image_year,comments,geometry
0,0,982,unknown Gilliam County,2,unknown,unknown,-99999,unknown,unknown,unknown,...,0,unknown,unknown,2013192,124.09,41-020577,Bing Maps Aerial,unknown,"FAA lists as Dismantle, google temporal analys...",POINT (-120.298786 45.653994)


In [13]:
wind_old_geo = gpd.GeoDataFrame(wind_old, 
                           crs = wind.crs, 
                           geometry = wind_old['geometry'])

In [14]:
wind_old_geo.columns

Index(['FID', 'unique_id', 'site_name', 'total_turb', 'on_year', 'year_range',
       'on_year_s', 'manufac', 'model', 'type_tower', 'decommiss',
       'MW_turbine', 'total_cpcy', 'total_ht', 'tower_h', 'blade_l',
       'rotor_dia', 'rotor_s_a', 'lat_DD', 'long_DD', 'state', 'county',
       'conf_attr', 'conf_loc', 'WENDI_name', 'EIA_name', 'FAA_jdate',
       'FAA_AGL', 'FAA_ORS', 'image_name', 'image_year', 'comments',
       'geometry'],
      dtype='object')

In [15]:
wind_old_geo = wind_old_geo.replace(-99999, np.nan)

In [16]:
wind_2014_by_year = wind_old_geo.groupby(['on_year_s']).unique_id.count().to_frame().reset_index()
wind_2014_by_year

,on_year_s,unique_id
0,1981.0,3679
1,1982.0,3635
2,1983.0,1343
3,1984.0,668
4,1985.0,602
5,1986.0,452
6,1987.0,264
7,1988.0,302
8,1989.0,440
9,1990.0,59


In [17]:
#create bins/year group column
bins= [1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015]
labels = ['1980-85','1985-90','1990-95','1995-00','2000-05','2005-10','2010-15']
wind_old_geo['year_group'] = pd.cut(wind_old_geo['on_year_s'], bins=bins, labels=labels, right=False)
wind_old_geo.head()

,FID,unique_id,site_name,total_turb,on_year,year_range,on_year_s,manufac,model,type_tower,...,WENDI_name,EIA_name,FAA_jdate,FAA_AGL,FAA_ORS,image_name,image_year,comments,geometry,year_group
0,0,982,unknown Gilliam County,2,unknown,unknown,NaN,unknown,unknown,unknown,...,unknown,unknown,2013192.0,124.09,41-020577,Bing Maps Aerial,unknown,"FAA lists as Dismantle, google temporal analys...",POINT (-120.29879 45.65399),NaN
1,1,1065,unknown Gilliam County,2,unknown,unknown,NaN,unknown,unknown,unknown,...,unknown,unknown,2013192.0,124.09,41-020578,Bing Maps Aerial,unknown,"FAA lists as Dismantle, google temporal analys...",POINT (-120.30599 45.66349),NaN
2,2,1895,Banner Wind Project,2,2008,no,2008.0,Entegrity,EW50,trestle,...,Banner Wind Project,unknown,2009065.0,38.72,02-020064,Bing Maps Aerial,unknown,"trestle turbines removed, new monopoles instal...",POINT (-165.42748 64.56990),2005-10
3,3,1897,Banner Wind Project,2,2008,no,2008.0,Entegrity,EW50,trestle,...,Banner Wind Project,unknown,2009065.0,38.72,02-020070,Bing Maps Aerial,unknown,"trestle turbines removed, new monopoles instal...",POINT (-165.42638 64.57090),2005-10
4,4,2608,unknown Tehachapi Wind Resource Area 1,549,1982_1990,yes,1982.0,unknown,unknown,monopole,...,unknown,unknown,NaN,NaN,unknown,NAIP,2012,,POINT (-118.32149 35.05990),1980-85


In [18]:
wind_old_geo.groupby('year_group').tower_h.mean()

year_group
1980-85    23.818189
1985-90    21.963264
1990-95    29.048558
1995-00    56.206355
2000-05    62.680189
2005-10    77.937642
2010-15    83.079573
Name: tower_h, dtype: float64

In [19]:
wind_old_geo.groupby('year_group').tower_h.max()

year_group
1980-85     36.5
1985-90     25.0
1990-95     43.0
1995-00     78.0
2000-05     85.0
2005-10    105.0
2010-15    120.0
Name: tower_h, dtype: float64

In [61]:
one = wind_old_geo[wind_old_geo['year_group'] == '1980-85']
two = wind_old_geo[wind_old_geo['year_group'] == '1985-90']

In [54]:
print(statistics.mode(list(one['tower_h'])))
print(statistics.mode(list(two['tower_h'])))

22.8
18.2


In [62]:
one = one.tower_h.value_counts().to_frame().reset_index()
one.columns = ['tower_height', 'count']

In [63]:
one

,tower_height,count
0,22.8,453
1,18.2,286
2,24.0,252
3,36.5,175
4,18.0,26
5,20.0,12


In [23]:
two.tower_h.value_counts()

18.2    440
24.0    304
24.6    210
23.0    116
25.0     96
24.5     17
24.3     12
Name: tower_h, dtype: int64

In [65]:
distribution = one['tower_height']
weights = one['count']

In [68]:
weight_avg = round(average(one['tower_height'], weights = one['count']), 2)

In [69]:
weight_avg

23.82

In [24]:
wind_old_geo.groupby('year_group').MW_turbine.max()

year_group
1980-85    0.40
1985-90    0.33
1990-95    0.36
1995-00    1.75
2000-05    3.00
2005-10    3.00
2010-15    3.60
Name: MW_turbine, dtype: float64

In [25]:
wind.groupby('p_year').t_hh.mean()

p_year
1981.0          NaN
1982.0          NaN
1983.0    22.800000
1984.0    24.000000
1985.0    24.386341
1986.0    25.285714
1987.0          NaN
1988.0    23.000000
1989.0          NaN
1990.0    30.000000
1991.0          NaN
1992.0    41.500000
1994.0    40.000000
1995.0    39.651163
1996.0    50.000000
1997.0    41.647059
1998.0    52.080645
1999.0    57.633149
2000.0    57.180556
2001.0    58.414576
2002.0    62.699782
2003.0    69.234783
2004.0    65.421779
2005.0    75.148028
2006.0    75.845436
2007.0    78.132560
2008.0    78.195780
2009.0    78.621082
2010.0    79.325188
2011.0    80.877153
2012.0    83.711371
2013.0    80.418803
2014.0    82.666133
2015.0    82.456762
2016.0    82.940776
2017.0    86.010600
2018.0    88.251646
2019.0    90.078615
2020.0    90.112127
Name: t_hh, dtype: float64

In [26]:
hub_height_year = wind.groupby(['p_year', 't_manu']).t_hh.mean().to_frame().reset_index()
hub_height_year.head()

,p_year,t_manu,t_hh
0,1982.0,Lolland,NaN
1,1982.0,Micon,NaN
2,1982.0,Vestas,NaN
3,1983.0,Vestas,22.8
4,1984.0,Bonus,24.0


In [27]:
statistics.mode(list(wind['t_hh']))

80.0

In [28]:
turbine_count_year = wind.groupby(['p_year', 't_manu']).case_id.count().to_frame().reset_index()

In [78]:
turbine_count_year = turbine_count_year.sort_values(by=['t_manu', 'p_year'])

In [82]:
turbine_count_year

,p_year,t_manu,case_id
127,2009.0,AAER,2
152,2010.0,AAER,1
29,1997.0,AOC,1
39,1999.0,AOC,5
46,2000.0,AOC,2
...,...,...,...
27,1995.0,Windmatic,1
232,2012.0,Windmatic,5
34,1997.0,Zond,12
38,1998.0,Zond,3


In [77]:
turbine_count_year.to_csv('turbine_year.csv')

In [30]:
wind.groupby(['p_year', 't_state']).case_id.count()

p_year  t_state
1981.0  CA          11
1982.0  CA         937
1983.0  CA         432
        OK           1
1984.0  CA         155
                  ... 
2020.0  TX         882
        VA           2
        WA          38
        WV          20
        WY         153
Name: case_id, Length: 502, dtype: int64

In [31]:
wind[wind['p_year'] == 1981.0].shape

(11, 18)

In [32]:
wind_v1 = pd.read_csv('../data/uswtdb_v1_0_20180419.csv')

In [33]:
wind_v1[wind_v1['p_year'] == 1981].shape

(537, 24)

In [34]:
wind_v1_year = wind_v1.p_year.value_counts().to_frame().reset_index().sort_values('index') #released in 2018
wind_v1_year.columns = ['year', 'count_v1']
year_install.columns = ['year', 'count_v3']

In [35]:
v1_v3 = pd.merge(wind_v1_year, year_install, on = 'year', how = 'right')
v1_v3

,year,count_v1,count_v3
0,1981,537.0,11
1,1982,1910.0,937
2,1983,979.0,433
3,1984,216.0,155
4,1985,1467.0,714
5,1986,311.0,151
6,1987,406.0,213
7,1988,282.0,277
8,1989,NaN,288
9,1990,432.0,327


In [36]:
#create bins/year group column
bins= [1980, 1985, 1990, 1995, 2000, 2005, 2010, 2015, 2020]
labels = ['1980-85','1985-90','1990-95','1995-00','2000-05','2005-10','2010-15', '2015-20']
wind['year_group'] = pd.cut(wind['p_year'], bins=bins, labels=labels, right=False)
wind.head()

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190),1985-90
1,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764),1985-90
2,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744),1985-90
3,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791),1985-90
4,3005333.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.368690,35.075294,POINT (-118.36869 35.07529),1985-90


In [37]:
wind.groupby('year_group').size()

year_group
1980-85     1536
1985-90     1643
1990-95      360
1995-00     1381
2000-05     3888
2005-10    17112
2010-15    16342
2015-20    18077
dtype: int64

In [38]:
wind_2000 = wind[wind['year_group'].isin(['2000-05', '2005-10', '2010-15', '2015-20'])]
wind_2000.year_group.value_counts()

2015-20    18077
2005-10    17112
2010-15    16342
2000-05     3888
1995-00        0
1990-95        0
1985-90        0
1980-85        0
Name: year_group, dtype: int64

In [39]:
wind_2000.groupby(['year_group', 't_state']).size().to_frame().reset_index().tail(50)

,year_group,t_state,0
310,2010-15,VT,44
311,2010-15,WA,556
312,2010-15,WI,113
313,2010-15,WV,151
314,2010-15,WY,185
315,2015-20,AK,9
316,2015-20,AR,0
317,2015-20,AZ,15
318,2015-20,CA,275
319,2015-20,CO,679


In [42]:
wind[wind['t_manu'] == 'Vestas']

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190),1985-90
1,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764),1985-90
2,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744),1985-90
3,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791),1985-90
4,3005333.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.368690,35.075294,POINT (-118.36869 35.07529),1985-90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
59728,3088836.0,OK,Wildhorse Mountain Wind Facility,2019.0,29.0,100.05,Vestas,V136-3.45,3450.0,105.0,136.0,14526.72,173.1,3.0,1.0,-95.242958,34.588802,POINT (-95.24296 34.58880),2015-20
60585,3090405.0,TX,Woodward Mountain II,2001.0,115.0,75.90,Vestas,V47-0.66,660.0,65.0,47.0,1734.94,88.4,3.0,3.0,-102.366089,30.966394,POINT (-102.36609 30.96639),2000-05
60628,3090403.0,TX,Woodward Mountain II,2001.0,115.0,75.90,Vestas,V47-0.66,660.0,65.0,47.0,1734.94,88.4,3.0,3.0,-102.364494,30.967396,POINT (-102.36449 30.96740),2000-05
60646,3090404.0,TX,Woodward Mountain II,2001.0,115.0,75.90,Vestas,V47-0.66,660.0,65.0,47.0,1734.94,88.4,3.0,3.0,-102.365288,30.966894,POINT (-102.36529 30.96689),2000-05


In [43]:
wind.columns

Index(['case_id', 't_state', 'p_name', 'p_year', 'p_tnum', 'p_cap', 't_manu',
       't_model', 't_cap', 't_hh', 't_rd', 't_rsa', 't_ttlh', 't_conf_atr',
       't_conf_loc', 'xlong', 'ylat', 'geometry', 'year_group'],
      dtype='object')

In [44]:
wind['rated_cap_mw'] = wind['t_cap']/1000

In [45]:
wind.head()

,case_id,t_state,p_name,p_year,p_tnum,p_cap,t_manu,t_model,t_cap,t_hh,t_rd,t_rsa,t_ttlh,t_conf_atr,t_conf_loc,xlong,ylat,geometry,year_group,rated_cap_mw
0,3005443.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.351089,35.091896,POINT (-118.35109 35.09190),1985-90,0.095
1,3072704.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364197,35.077644,POINT (-118.36420 35.07764),1985-90,0.095
2,3072695.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.364410,35.077435,POINT (-118.36441 35.07744),1985-90,0.095
3,3072661.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.363762,35.077908,POINT (-118.36376 35.07791),1985-90,0.095
4,3005333.0,CA,251 Wind,1987.0,194.0,18.43,Vestas,None,95.0,NaN,NaN,NaN,NaN,2.0,3.0,-118.368690,35.075294,POINT (-118.36869 35.07529),1985-90,0.095


In [70]:
wind.groupby('year_group').t_hh.mean()

year_group
1980-85    22.853097
1985-90    24.096277
1990-95    30.363905
1995-00    55.616766
2000-05    62.694380
2005-10    77.833081
2010-15    82.033483
2015-20    85.702717
Name: t_hh, dtype: float64

In [72]:
wind.groupby('year_group').t_hh.max()

year_group
1980-85     24.0
1985-90     60.0
1990-95     43.0
1995-00     80.0
2000-05     80.0
2005-10    105.0
2010-15    103.0
2015-20    131.0
Name: t_hh, dtype: float64